In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import time
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer
import torch
import re
import string

import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [6]:
rslt = 'https://www.tripadvisor.in/Restaurant_Review-g580418-d26134428-Reviews-Adyar_Ananda_Bhavan_A2B-Harrow_Greater_London_England.html'

In [32]:
import random
import time
import requests
from bs4 import BeautifulSoup

user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edge/87.0.664.75',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363'
]

def getAndParseURL(url):
    session = requests.Session()  # Start a session
    headers = {
        "User-Agent": user_agents[random.randint(0, len(user_agents)-1)],
    }
    result = session.get(url, headers=headers)  # Use the session to get the page
    soup = BeautifulSoup(result.text, 'html.parser')
    time.sleep(random.uniform(1.0, 2.0))  # Add delay between requests
    return soup

In [33]:
result = []

html = getAndParseURL(rslt)

In [ ]:
html

In [35]:
try:
    div_elements = html.find_all("div", {"class": "entry"})
    time.sleep(1)
    # print(div_elements)

    reviews = [div.find("p", class_="partial_entry").text for div in div_elements]
    time.sleep(1)
    # print(reviews)
    
except:
    print("Except block is called!")
    reviews = [np.nan]

result.extend([reviews])

In [36]:
result

[['It was our first visit to this restarant after trying many other South Indian restaurants in and around Rayners Lane Eastcote,Harrow, Wembley and Hounsow. The food and decor was acceptable but the service was terrible! It was Sunday evening and not partularly busy and yet...one of the orders for uttampam was 45 minutes!! THe waitress made very lame excuses saying it takes longer to cook but not 45 minutes! She tried to fob us off istead of finding out from kitchen for the delay - ultimately she did tht but it was too late for us as we had waited far too long. what was to be an enjoyable evening, it turned out to be a terrible one! Iam afraid non-of us will be visiting this restaurant again! There are plenty of good restaurants  locally with much better service!More',
  "The food is nice. Tasty. The menu doesnt really explain much so I don't think they are catering for a wide audience.  Food price is fair but price of non-alcoholic drinks is outrageous. The quantity of the meals was 

In [37]:
reviews = result[0]

In [38]:
reviews

['It was our first visit to this restarant after trying many other South Indian restaurants in and around Rayners Lane Eastcote,Harrow, Wembley and Hounsow. The food and decor was acceptable but the service was terrible! It was Sunday evening and not partularly busy and yet...one of the orders for uttampam was 45 minutes!! THe waitress made very lame excuses saying it takes longer to cook but not 45 minutes! She tried to fob us off istead of finding out from kitchen for the delay - ultimately she did tht but it was too late for us as we had waited far too long. what was to be an enjoyable evening, it turned out to be a terrible one! Iam afraid non-of us will be visiting this restaurant again! There are plenty of good restaurants  locally with much better service!More',
 "The food is nice. Tasty. The menu doesnt really explain much so I don't think they are catering for a wide audience.  Food price is fair but price of non-alcoholic drinks is outrageous. The quantity of the meals was go

In [41]:
url_pattern = re.compile(r'https?://\S+|www\.\S+')
punctuation_pattern = re.compile('[%s]' % re.escape(string.punctuation))
newline_pattern = re.compile('\n')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
tag_dict = {'J': wordnet.ADJ,'N': wordnet.NOUN,'V': wordnet.VERB,'R': wordnet.ADV}

In [42]:
def preprocess_text(text):
    text = str(text).lower()
    clean_text = re.sub('https?://\S+|www\.\S+', '', text) #URLs
    clean_text = re.sub('[%s]' % re.escape(string.punctuation), '', clean_text) #Punctuations
    clean_text = re.sub('\n', '', clean_text) #Backslash n
    clean_text = [word for word in clean_text.split(' ') if word not in stop_words] #Remove stopwords
    pos_tags = nltk.pos_tag(clean_text)
    wordnet_tags = [(token, tag_dict.get(tag[0].upper(), wordnet.NOUN)) for token, tag in pos_tags]
    tokens = [lemmatizer.lemmatize(token, tag) for token, tag in wordnet_tags] #Lemmatize
    return " ".join(tokens)

In [43]:
clean_review = [preprocess_text(review) for review in reviews]

In [45]:
clean_review

['first visit restarant try many south indian restaurant around rayners lane eastcoteharrow wembley hounsow food decor acceptable service terrible sunday even partularly busy yetone order uttampam 45 minute waitress make lame excuse say take long cook 45 minute try fob u istead find kitchen delay  ultimately tht late u wait far long enjoyable evening turn terrible one iam afraid nonof u visit restaurant plenty good restaurant  locally much good servicemore',
 'food nice tasty menu doesnt really explain much dont think cater wide audience  food price fair price nonalcoholic drink outrageous quantity meal good  service isnt good',
 'order lunch 30 people respond call message discuss kalyani order closer actual date totally vanish stop communicate order elsewhere last minute willnot recommend a2bmore',
 'soon saw new restaurant go inside table provide immediately  serve u take 12 hour order take another 12 hour food even think leave go tosaravanabhavan usual restaurant girl pleasant time 

In [46]:
model_dir = "RoBERTa_weights"
model_path = f"{model_dir}/model.pth"
tokenizer_path = f"{model_dir}/tokenizer"

In [47]:
class Roberta(torch.nn.Module):
    def __init__(self):
        super(Roberta, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.2)
        self.classifier = torch.nn.Linear(768, 5)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [48]:
model = Roberta()

model.load_state_dict(torch.load(model_path))

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [49]:
tokenizer = RobertaTokenizer.from_pretrained(tokenizer_path)

In [50]:
previews = []
nreviews = []

In [51]:
for i in range(len(clean_review)):
     inputs = tokenizer(clean_review[i], return_tensors="pt", padding=True, truncation=True)
     input_ids = inputs['input_ids']
     attention_mask = inputs['attention_mask']

     with torch.no_grad():
          outputs = model(input_ids, attention_mask, None)

     probs = torch.nn.functional.softmax(outputs, dim=-1)
     predicted_label = torch.argmax(probs, dim=-1).item()
     if (predicted_label == 0):
          nreviews.append(reviews[i])
     else:
          previews.append(reviews[i])

In [52]:
previews

['I had a combinations of South Indian breakfast..The food was not bad but also not what we were craving for. But got an exceptional service from Rutvi from beginning to the end, she was very polite and helpful.',
 'Great food! But Krinal was the start of the visit!! \nThank you krnial for making it an amazing experience ❤️❤️',
 'Came here recently for dinner for the first time, having heard about the new restaurant. The place has a nice vibe and is decorated nicely inside. We were seated quickly, and service was fast and friendly. Lots of menu options, and all the dishes we...chose tasted great! Prices seemed reasonable for the quality and portion size too. So overall, a good first impression, no complaints!More',
 'Food is great, service is quick and helpful. We were served by Rutvika and Rupali who were really kind and explained everything to us. \n\nWould recommend',
 'Excellent service,  tasty dishes we enjoyed all fresh, hot dishes piping hot and joy to eat. Catered to my allergy

In [53]:
nreviews

['It was our first visit to this restarant after trying many other South Indian restaurants in and around Rayners Lane Eastcote,Harrow, Wembley and Hounsow. The food and decor was acceptable but the service was terrible! It was Sunday evening and not partularly busy and yet...one of the orders for uttampam was 45 minutes!! THe waitress made very lame excuses saying it takes longer to cook but not 45 minutes! She tried to fob us off istead of finding out from kitchen for the delay - ultimately she did tht but it was too late for us as we had waited far too long. what was to be an enjoyable evening, it turned out to be a terrible one! Iam afraid non-of us will be visiting this restaurant again! There are plenty of good restaurants  locally with much better service!More',
 "The food is nice. Tasty. The menu doesnt really explain much so I don't think they are catering for a wide audience.  Food price is fair but price of non-alcoholic drinks is outrageous. The quantity of the meals was go

In [54]:
preview = '. '.join(previews)

In [56]:
nreview = '. '.join(nreviews)

In [59]:
from summarizer import Summarizer
summarizer = Summarizer()

### Summary of Negative Reviews

In [ ]:
snr = summarizer(nreview, num_sentences=5)

In [57]:
len(nreview)

3523

In [66]:
len(snr)

498

In [62]:
snr

'It was our first visit to this restarant after trying many other South Indian restaurants in and around Rayners Lane Eastcote,Harrow, Wembley and Hounsow. When l asked she said ‘we don’t provide’!!! Overall very poor customer service and not value for money. We ordered chat and dosa and the waitress brought the dosa first and when asked she said that she forgot without any apologies. I got the Onam special Meal £23.95\nHonestly it was the worst meal and so expensive\nStaff need lots of training.'

### Summary of Positive Reviews

In [ ]:
spr = summarizer(preview)

In [58]:
len(preview)

1022

In [67]:
len(spr)

285

In [65]:
spr

'I had a combinations of South Indian breakfast..The food was not bad but also not what we were craving for. But got an exceptional service from Rutvi from beginning to the end, she was very polite and helpful.. Great food! Prices seemed reasonable for the quality and portion size too.'